<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importando-dataframe" data-toc-modified-id="Importando-dataframe-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importando dataframe</a></span><ul class="toc-item"><li><span><a href="#Normalizando-o-dado" data-toc-modified-id="Normalizando-o-dado-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Normalizando o dado</a></span></li><li><span><a href="#Separando-treino-e-teste" data-toc-modified-id="Separando-treino-e-teste-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Separando treino e teste</a></span></li><li><span><a href="#Dados-balanceados" data-toc-modified-id="Dados-balanceados-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Dados balanceados</a></span></li></ul></li></ul></div>

# Separação dos dados de treino e teste

In [1]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH, MODEL_PATH

import os
from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import pandas_profiling

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Metrics
from plot_metrics import plot_roc, plot_confusion

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

colorscale = ['#025951', '#8BD9CA', '#BF7F30', '#F2C124', '#8C470B', '#DFC27D']

## Importando dataframe

In [2]:
df = pd.read_csv(TREAT_PATH / 'analysis' / 'df_alunos.csv', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182838 entries, 0 to 182837
Data columns (total 19 columns):
ID                            182838 non-null int64
CO_ENTIDADE                   182838 non-null int64
TP_SEXO                       182838 non-null int64
NU_IDADE                      182838 non-null int64
IN_DISTORCAO                  182838 non-null int64
IN_TRANSPORTE_PUBLICO         182838 non-null int64
IN_LOCAL_ESCOLA               182838 non-null int64
N_TURMA                       182838 non-null int64
IN_EVASAO                     182838 non-null int64
NU_ANO_CENSO                  182838 non-null int64
CO_MUNICIPIO                  182838 non-null int64
IN_LABORATORIO_INFORMATICA    182838 non-null float64
IN_LABORATORIO_CIENCIAS       182838 non-null float64
IN_QUADRA_ESPORTES            182838 non-null float64
IN_BIBLIOTECA                 182838 non-null float64
IN_BANHEIRO_FORA_PREDIO       182838 non-null float64
IN_AREA_VERDE                 182838 non-null f

In [3]:
df = df.drop('NU_ANO_CENSO', axis=1)
df.head(2)

,ID,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,IN_EVASAO,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL
0,112132289130,33009627,2,17,0,0,2,0,0,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
1,113220290328,33009627,2,21,1,0,2,0,1,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0


In [4]:
df['IN_EVASAO'].value_counts(normalize=True)

0    0.657172
1    0.342828
Name: IN_EVASAO, dtype: float64

In [5]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149234 entries, 0 to 182836
Data columns (total 18 columns):
ID                            149234 non-null int64
CO_ENTIDADE                   149234 non-null int64
TP_SEXO                       149234 non-null int64
NU_IDADE                      149234 non-null int64
IN_DISTORCAO                  149234 non-null int64
IN_TRANSPORTE_PUBLICO         149234 non-null int64
IN_LOCAL_ESCOLA               149234 non-null int64
N_TURMA                       149234 non-null int64
IN_EVASAO                     149234 non-null int64
CO_MUNICIPIO                  149234 non-null int64
IN_LABORATORIO_INFORMATICA    149234 non-null float64
IN_LABORATORIO_CIENCIAS       149234 non-null float64
IN_QUADRA_ESPORTES            149234 non-null float64
IN_BIBLIOTECA                 149234 non-null float64
IN_BANHEIRO_FORA_PREDIO       149234 non-null float64
IN_AREA_VERDE                 149234 non-null float64
IN_N_COMP_15                  149234 non-null

In [6]:
df['IN_EVASAO'].value_counts(normalize=True)

0    0.676689
1    0.323311
Name: IN_EVASAO, dtype: float64

In [7]:
# Unicidade do ID
df['ID'].value_counts()[:5]

111677686633    1
116834315887    1
112387438468    1
144787363408    1
144782776569    1
Name: ID, dtype: int64

In [8]:
df = df.set_index('ID', drop=True)

In [9]:
X = df.columns
X = X.drop('IN_EVASAO')
df_x = df[X]

df_x.head(2)

,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL
ID,,,,,,,,,,,,,,,,
112132289130,33009627,2,17,0,0,2,0,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0
113220290328,33009627,2,21,1,0,2,0,3301009,1.0,1.0,1.0,1.0,1.0,0.0,1,3.0


In [10]:
y = df['IN_EVASAO']
y.head()

ID
112132289130    0
113220290328    1
112752673352    0
117459545545    0
113868062938    1
Name: IN_EVASAO, dtype: int64

### Normalizando o dado

Normalização das variáveis contínuas do dado.

In [28]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
X_norm = enc.fit_transform(df_x)

# # Salvando encoder
# import pickle
# filename = 'ordinal_encoder.sav'
# pickle.dump(enc, open(MODEL_PATH / filename, 'wb'))

import pickle
filename = 'ordinal_encoder.sav'
enc = pickle.load(open(MODEL_PATH / filename,"rb"))

In [29]:
enc.categories_

[array([33000026, 33000042, 33000158, 33000360, 33000670, 33001014,
        33001022, 33001278, 33001286, 33001294, 33001316, 33001332,
        33001359, 33001456, 33001650, 33002126, 33002258, 33002312,
        33002614, 33002827, 33003386, 33003394, 33003424, 33003858,
        33004226, 33004447, 33004749, 33004943, 33005362, 33005478,
        33005664, 33005990, 33006296, 33006458, 33006571, 33006660,
        33006679, 33006709, 33006776, 33007101, 33007306, 33007403,
        33007420, 33007900, 33008272, 33008299, 33008370, 33008388,
        33008396, 33008418, 33008434, 33008442, 33008450, 33008485,
        33008493, 33008663, 33008728, 33008736, 33008752, 33008795,
        33008850, 33008868, 33008892, 33008981, 33009007, 33009120,
        33009627, 33009775, 33010110, 33010170, 33010269, 33010528,
        33010889, 33011095, 33011400, 33011427, 33011435, 33011460,
        33011621, 33011842, 33012040, 33012083, 33012121, 33012393,
        33012407, 33012458, 33013160, 33013322, 

In [13]:
X_norm = pd.DataFrame(X_norm, columns = df_x.columns, index=df_x.index)
X_norm.head(2)

,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL
ID,,,,,,,,,,,,,,,,
112132289130,66.0,1.0,5.0,0.0,0.0,2.0,0.0,14.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0
113220290328,66.0,1.0,9.0,1.0,0.0,2.0,0.0,14.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0


In [14]:
X_norm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149234 entries, 112132289130 to 112980381895
Data columns (total 16 columns):
CO_ENTIDADE                   149234 non-null float64
TP_SEXO                       149234 non-null float64
NU_IDADE                      149234 non-null float64
IN_DISTORCAO                  149234 non-null float64
IN_TRANSPORTE_PUBLICO         149234 non-null float64
IN_LOCAL_ESCOLA               149234 non-null float64
N_TURMA                       149234 non-null float64
CO_MUNICIPIO                  149234 non-null float64
IN_LABORATORIO_INFORMATICA    149234 non-null float64
IN_LABORATORIO_CIENCIAS       149234 non-null float64
IN_QUADRA_ESPORTES            149234 non-null float64
IN_BIBLIOTECA                 149234 non-null float64
IN_BANHEIRO_FORA_PREDIO       149234 non-null float64
IN_AREA_VERDE                 149234 non-null float64
IN_N_COMP_15                  149234 non-null float64
NIVEL                         149234 non-null float64
dtypes: 

### Separando treino e teste

In [15]:
# from sklearn.model_selection import train_test_split 
# X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=0)

# # Salvando treino e teste
# X_train.join(y_train).to_csv(TREAT_PATH / 'modelo' / 'df_treino.csv')
# X_test.join(y_test).to_csv(TREAT_PATH / 'modelo' / 'df_teste.csv')

X_train = pd.read_csv(TREAT_PATH / 'modelo' / 'df_treino.csv', index_col='ID')
y_train = X_train['IN_EVASAO']
X_train = X_train.drop('IN_EVASAO', axis=1)

X_test = pd.read_csv(TREAT_PATH / 'modelo' / 'df_teste.csv', index_col='ID')
y_test = X_test['IN_EVASAO']
X_test = X_test.drop('IN_EVASAO', axis=1)

In [16]:
X_train.head(2)

,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL
ID,,,,,,,,,,,,,,,,
116560246166,372.0,1.0,3.0,0.0,0.0,1.0,1.0,7.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0
117459039844,816.0,1.0,4.0,0.0,1.0,0.0,1.0,64.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0


In [17]:
y_train.head(2)

ID
116560246166    0
117459039844    0
Name: IN_EVASAO, dtype: int64

In [18]:
y_train.value_counts(normalize=True)

0    0.676507
1    0.323493
Name: IN_EVASAO, dtype: float64

In [19]:
X_test.head(2)

,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL
ID,,,,,,,,,,,,,,,,
112810788919,311.0,1.0,4.0,0.0,1.0,2.0,0.0,81.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0
112378190408,830.0,0.0,3.0,0.0,1.0,0.0,0.0,64.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0


In [20]:
y_test.head(2)

ID
112810788919    0
112378190408    0
Name: IN_EVASAO, dtype: int64

In [21]:
y_test.value_counts(normalize=True)

0    0.677112
1    0.322888
Name: IN_EVASAO, dtype: float64

### Dados balanceados

Problema dos dados desbalanceados: tendência para predição da classe dominante é maior, pois aumenta a acurácia do modelo - mas não necessariamente o *recall*, que é o nosso foco.


[Rebalanceamento](https://towardsdatascience.com/having-an-imbalanced-dataset-here-is-how-you-can-solve-it-1640568947eb):

- Undersampling: remoção aleatória de observações da classe dominante

- Oversampling: geração de observações artificiais da classe minoritária
 - SMOTE (Synthetic Minority Over-sampling Technique): k nearest neighbours
 
Como temos muito dado, vamos escolher uma amostra aleatória da classe dominante para usar no treinamento do modelo.

In [22]:
from sklearn.utils import resample

df_maj = X_train[X_train.index.isin(y_train[y_train == 0].index)]
df_min = X_train[X_train.index.isin(y_train[y_train == 1].index)]

In [23]:
print(len(df_maj), len(df_min))

70670 33793


In [24]:
# # Selecionando uma amostra da classe majoritária
# df_maj = df_maj.sample(n=len(df_min),random_state=0)

# X_train_bal = df_min.append(df_maj)
# y_train_bal = y_train[y_train.index.isin(X_train_bal.index)]

# # Salvando treino balanceado
# X_train_bal.join(y_train_bal).to_csv(TREAT_PATH / 'modelo' / 'df_treino_bal.csv')

X_train_bal = pd.read_csv(TREAT_PATH / 'modelo' / 'df_treino_bal.csv', index_col='ID')
y_train_bal = X_train_bal['IN_EVASAO']
X_train_bal = X_train_bal.drop('IN_EVASAO', axis=1)

In [25]:
X_train_bal.head(2)

,CO_ENTIDADE,TP_SEXO,NU_IDADE,IN_DISTORCAO,IN_TRANSPORTE_PUBLICO,IN_LOCAL_ESCOLA,N_TURMA,CO_MUNICIPIO,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BANHEIRO_FORA_PREDIO,IN_AREA_VERDE,IN_N_COMP_15,NIVEL
ID,,,,,,,,,,,,,,,,
110726113092,837.0,0.0,3.0,0.0,1.0,0.0,2.0,64.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0
144787291237,750.0,0.0,3.0,0.0,1.0,1.0,1.0,26.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0


In [26]:
y_train_bal.head(2)

ID
110726113092    1
144787291237    1
Name: IN_EVASAO, dtype: int64

In [27]:
y_train_bal.value_counts()

1    33793
0    33793
Name: IN_EVASAO, dtype: int64